<a href="https://colab.research.google.com/github/zzhining/stock_market_analysis/blob/main/3%EC%A3%BC%EC%B0%A8/%EC%B6%94%EC%84%B8_%EB%A7%A4%EB%A7%A4_%EC%A0%84%EB%9E%B5_%EC%A0%81%EC%9A%A9%ED%95%98%EA%B8%B0.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 환경설정

In [ ]:
 !pip install pykrx

In [ ]:
# (라이브러리 설치 후 런타임 재시작 필요)
!sudo apt-get install -y fonts-nanum
!sudo fc-cache -fv
!rm ~/.cache/matplotlib -rf

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px

# Matplotlib 한글 폰트 설정
plt.rc('font', family='NanumBarunGothic')   

-----

# 이동평균(MA, Moving Average)이란?

- 일정한 기간 동안의 종가를 평균으로 나타낸 것
- 예) 20선: 20일 동안의 평균 종가
- 주로 20일, 50일, 200일 이동 평균을 활용


## 이동 평균의 활용


### 지지영역, 저항영역 분석

- 주가는 평균에서 너무 멀리 벗어나면 다시 평균값으로 환원하려는 경향이 있음
- 이동 평균선은 주가 추세가 일정 구간을 벗어나려고 할 때 제동을 거는 장벽 역할을 하므로,  지지영역, 저항영역을 분석하는데 활용



### 현 추세의 방향과 강도를 결정

- 추세의 방향
    - 이동 평균이 ‘**상승곡선**’일 경우: 주가의 추세도 상승
    - 이동 평균이 **‘하락곡선’**일 경우: 주가의 추세도 하락
    - 이동 평균이 **‘수평이거나 오르내림을 반복**’할 경우: 시장이 박스권 등락 국면임
- 추세의 강도
    - 경사가 가파르면 추세가 강하고, 경사가 비스듬하면 추세가 약하다.

---

## 주가 가져오기

In [ ]:
from pykrx import stock

start_date = "20210101"
end_date = "20230412"
today = "20230412"
ticker = "373220" # LG에너지 솔루션
df = stock.get_market_ohlcv(start_date, end_date, ticker)
df.head(3)

## 이동평균 구하기

In [ ]:
df['ma5'] = df['종가'].rolling(5).mean()
df['ma20'] = df['종가'].rolling(20).mean()
df['ma100'] = df['종가'].rolling(100).mean()
df['ma200'] = df['종가'].rolling(200).mean()

In [ ]:
df.head()

## 이동평균 시각화

In [ ]:
import plotly.express as px
df1 = df[['종가', 'ma5', 'ma20', 'ma100', 'ma200']]
fig = px.line(df1,  y=df1.columns,
              title='이동평균선')
fig.update_xaxes(
    dtick="M1",
    tickformat="%b\n%Y",
    ticklabelmode="period")
fig.show()

# [스토캐스틱(Stochastic)]( https://ko.wikipedia.org/wiki/%EC%8A%A4%ED%86%A0%EC%BA%90%EC%8A%A4%ED%8B%B1)
- 최근 N일간의 최고가와 최저가의 범위 내에서 현재 가격의 위치를 표시
- **매수세** > 매도세: 현재 가격의 위치가 높게 형성됨
- **매도세** > 매수세: 현재 가격의 위치가 낮게 형성됨

$$스토캐스틱N = \frac{(현재 가격 - N일 중 최저가)}{(N일 중 최고가 - N일중 최저가)}$$

- 스토캐스틱 값의 범위는 항상 0~100% 사이
    - 스토캐스틱 = **100%**: 현재 가격이 N일간 최고가, 매수세가 강함
    - 스토캐스틱 = **0%**: 현재 가격이 N일간 최저가, 매도세가 강함함
    - 스토캐스틱 >= **80%**: 과매수 상태(하락 가능성이 커짐),
    - 스토캐스틱 <= **20%**: 과매도 상태(상승 가능성이 커짐)
    

----
(예)
```
최근 5일간 최고가가 15,000원이고 최저가가 10,000원인 주식
```
- 현재가가 14,000원이라면?
    - 매수세가 강하여 오르는 추세임
    - 스토캐스틱 값은 80%
- 현재가가 11,000원이라면?
    - 매도세가 강하여 내리는 추세임
    - 스토캐스틱 값은 20%
----

## Fast Stochastic 공식


$$Fast \%K = \frac{(현재 가격 - N일 중 최저가)}{(N일 중 최고가 - N일중 최저가)}$$
<br>
$$Fast \%D = Fask \%K의 m기간 이동평균$$
<br>
- 일반적으로 N=5. m=3을 사용
- N을 10으로 사용할 경우, m=6으로 사용
 



## Slow Stochastic 공식

$$Slow\%K = Fast\%K의 m기간 이동평균$$
<br>
$$Slow\%D = Slow\%K의 t기간 이동평균$$
<br>

- Slow Stochastic에서는 n(5)-m(3)-t(3) 공식이 가장 많이 사용
- 네이버금융은 n(15)-m(5)-t(3)을 사용

- Fast Stochastic은 그래프의 변화가 너무 잦고 급격하여 노이즈 즉 가짜 신호가 많아 매수 매도시 참고하기 어렵다는 문제점이 있기 때문에 일반적으로 Slow Stochastic을 사용

 


 

## 스토캐스틱 구하기

In [ ]:
# Fast %K = ((현재가 - n기간 중 최저가) / (n기간 중 최고가 - n기간 중 최저가)) * 100
def get_stochastic_fast_k(close_price, low, high, n=5):
  fast_k = ((close_price - low.rolling(n).min()) / (high.rolling(n).max() - low.rolling(n).min())) * 100
  return fast_k
 
# Slow %K = Fast %K의 m기간 이동평균
def get_stochastic_slow_k(fast_k, n=3):
  slow_k = fast_k.rolling(n).mean()
  return slow_k
 
# Slow %D = Slow %K의 t기간 이동평균
def get_stochastic_slow_d(slow_k, n=3):
  slow_d = slow_k.rolling(n).mean()
  return slow_d 

스토캐스틱 구하기

In [ ]:
# fast_k, slow_k, slow_d를 획득
df['fast_k'] = get_stochastic_fast_k(df['종가'], df['저가'], df['고가'], 5)
df['slow_k'] = get_stochastic_slow_k(df['fast_k'], 3)
df['slow_d'] = get_stochastic_slow_d(df['slow_k'], 3)

## 스토캐스틱 시각화

In [ ]:
from plotly.subplots import make_subplots
import plotly.graph_objects as go

# Create figure with secondary y-axis
fig = make_subplots(specs=[[{"secondary_y": True}]])

fig.add_trace(
    go.Line(x = df.index, y = df['종가'], name='종가'),
    secondary_y= True
)

# Add traces
fig.add_trace(
    go.Line(x = df.index, y = df['slow_k'], name='slow_k'),
    secondary_y= False
)

# Add traces
fig.add_trace(
    go.Line(x = df.index, y = df['slow_d'], name='slow_d'),
    secondary_y= False
)

# Add traces
fig.add_trace(
    go.Line(x = df.index, y=[80] * len(df.index), name='80선'),
    secondary_y= False
)

# Add traces
fig.add_trace(
    go.Line(x = df.index, y=[20] * len(df.index), name='20선'),
    secondary_y= False
)

fig.update_xaxes(
    rangeslider_visible=True,
    rangeselector=dict(
        buttons=list([
            dict(count=1, label="1m", step="month", stepmode="backward"),
            dict(count=6, label="6m", step="month", stepmode="backward"),
            dict(count=1, label="YTD", step="year", stepmode="todate"),
            dict(count=1, label="1y", step="year", stepmode="backward"),
            dict(step="all")
        ])
    )
)

# Set y-axes titles
fig.update_yaxes(title_text="<b>stocastic</b> axis", secondary_y=False)
fig.update_yaxes(title_text="<b>종가</b> axis", secondary_y=True)

fig.show()

## 골든크로스
- 단기이동평균선이 장기이동평균선을 뚫고 상승하는 것
- 스토캐스틱 20 이하에서 %K선이 %D선을 상향 돌파
- 매수 시점

In [ ]:
df.query('(slow_k >= slow_d) and (slow_k <= 20)')

## 데드크로스
- 단기이동평균선이 장기이동평균선을 뚫고 하락하는 것
- 스토캐스틱 80 이상상에서 %K선이 %D선을 하향 돌파
- 매도 시점


In [ ]:
df.query('(slow_k <= slow_d) and (slow_k >= 80)')